# Introduction

Hopefully the [Usage Demo Notebook](./usage_demo.ipynb) made you comfortable in using the `look_up` function. Now, this notebook will have demo plots that I generally use in design using ${g_{m}}\over{i_{d}}$ methodology which will henceforth be typed as gm_id methodology. This methodology treats gm_id as an independent variable in addition to Length and bias current. All of them when selected appropriately should give you a transistor biased to be used as an active device.

This is just one of the templates that I use and you can create a template of your choice and use your methodology of choice. For example, instead of gm/id as an independent variable you use [inversion coefficient as the independent variable](https://kevinfronczak.com/blog/inversion-coefficient-based-circuit-design) or any other.

However, at the core of any methodology is the Look up Table (LUT) you will access and hence the need to iterate device dimensions to achieve the correct biasing should be minimal.

Note: These plots will always be W.I.P, hence if you don't like certain changes you can always access the historical versions!

The following code block essentially imports/includes the necessary libraries. Additionally, it tries to import the config file named `conf.py`. You defined the values in this file in [Import Demo](./import_demo.ipynb). Do not attempt to write this file manually. If you want to rewrite head back to the relevant section of [Import Demo](./import_demo.ipynb). Make sure to append the correct library path to the system path. In case you are using the `docs/notebooks` directory `../../` is your library path.

In [2]:
import sys
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
# Append the path of your library to the system path
sys.path.append(os.path.abspath("../../"))
from analog_daddy.look_up import look_up
from analog_daddy.utils import pretty_print_structure, describe_structure, look_upW, look_upW_circuit, print_circuit
from analog_daddy.conf import * # import all the config variables

In [3]:
DATABASE_ROOT_PATH = "../../ignored_folder/databases/"
PDK_NAME = "freepdk45"
FILE_NAME = "nom_25.npy"
DATABASE_LUT_PATH = os.path.join(DATABASE_ROOT_PATH, PDK_NAME, "NPY", FILE_NAME)
z = np.load(DATABASE_LUT_PATH, allow_pickle=True).item()
nmos = z['nmos_vth']
pmos = z['pmos_vth']

In [4]:
# You can use the following to print the structure of device if you get confused
# about the keys or min and max values in the conf file.
# pretty_print_structure(describe_structure(nmos))

## Generic Code

Life is too short to change specifics as well as keep on creating redundant templates.
You can change `outvar` to `gm_cgg`,`gds_id` or any valid ratio.
Uncomment `length_sweep` variables to get get usable graphs for design and to better observe tradeoffs. 

In [5]:
outvar = "gds_gds"
gm_id_sweep = np.linspace(5,25,15)
length_sweep = np.linspace(L_MIN, 500e-09, 5)
# length_sweep = np.linspace(L_MIN, L_MAX, 15)
# length_sweep = np.linspace(500e-09, L_MAX, 15)

y = np.array([look_up(nmos, outvar, gm_id=gm_id_sweep, length=length) for length in length_sweep])
fig = go.Figure()

for idx, length in enumerate(length_sweep):
    fig.add_trace(go.Scatter(x=gm_id_sweep, y=y[idx], mode='lines', name=f'L={length} m'))

fig.update_layout(title=f'Parametric Plot of {outvar} vs gm_id for different lengths', xaxis_title='gm_id', yaxis_title=f'{outvar}')
fig.show()


# Design Process

Use the above plots or key important params and decide a value of gm_id and length.
To get width, fix your bias current value. Define a structure and give name to your transistor. Give something
which is logical. Instead of M1, M2 use GM_PAIR, NMOS_LOAD, NMOS_CASC et cetera.

In [6]:
five_t_ota = {
                "gm_pair":
                {
                    "type": nmos,
                    "length": L_MIN,
                    "gm_id" : 15,
                    "id": 10e-6,
                    "w": W_MIN,
                    "description": "Just in case you want to be more detailed."
                },

                "diode_pair":
                {
                    "type": pmos,
                    "length": L_MIN,
                    "gm_id" : 15,
                    "id": 10e-6,
                    "w": W_MIN,
                    "description": "Just in case you want to be more detailed."
                },

            }


In [7]:
if look_upW_circuit(five_t_ota):
    print_circuit(five_t_ota)
else:
    print("Issue with the circuit.")


----------BEGIN--------------------


-----------------------------------

Type: Refer dictionary.
length:4.5e-08
gm_id:15
id:1e-05
w:2.496702132034059e-07
description:Just in case you want to be more detailed.

-----------------------------------


-----------------------------------

Type: Refer dictionary.
length:4.5e-08
gm_id:15
id:1e-05
w:9e-08
description:Just in case you want to be more detailed.

-----------------------------------


------------END--------------------

